In [ ]:
# CELDA 1: imports and mqtt configuration

import time
import numpy as np
from datetime import datetime

from fmi_mlc import fmi_gym
from mqtt_publisher import MQTTPublisher
import paho.mqtt.client as mqtt
from config import parameter as dfmu

BROKER_HOST = "mosquitto"
BROKER_PORT = 1883

# fmu parameters
action_names = dfmu['action_names']
action_min   = dfmu['action_min']

# 1) Publisher creation for observations and actions
pub = MQTTPublisher(host=BROKER_HOST, port=BROKER_PORT)

# 2) Dictionary to save latest action
latest_actions = { name: None for name in action_names }

# 3) Client for subscribing to openhab commands
client = mqtt.Client()

def on_connect(client, userdata, flags, rc):
    print(f"[{datetime.now().strftime('%H:%M:%S')}] [SIM] connected to broker {rc}. Susbscribing to topics…")
    for name in action_names:
        topic = f"openhab/actuator/{name}" #this is just for this example, adjust the topics as you configured them in openhab 
        client.subscribe(topic, qos=1)
        print(f"    • [SIM] Subscribed to {topic}")

def on_message(client, userdata, msg):
    try:
        value = float(msg.payload.decode())
    except:
        print(f"[{datetime.now().strftime('%H:%M:%S')}] [SIM] Payload no numérico: {msg.payload!r}")
        return
    action_key = msg.topic.split("/")[-1]
    latest_actions[action_key] = value
    print(f"[{datetime.now().strftime('%H:%M:%S')}] [SIM] Command → {action_key} = {value}") #just for debug, not necessary

client.on_connect = on_connect
client.on_message = on_message

# 4) Coonect and start loop for mqtt command subscriber
client.connect(BROKER_HOST, BROKER_PORT)
client.loop_start()

# 5) FMU gym environment inizialization
env = fmi_gym(dfmu)
initial_obs = env.reset()
print("► [SIM] fmu initialized, first observation:", initial_obs)

C:\Users\andre\AppData\Local\Temp\ipykernel_29460\3221505377.py:26: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


[09:09:45] [SIM] connected to broker 0. Susbscribing to topics…
    • [SIM] Subscribed to openhab/actuator/Heat_SP_Kitchen_fmu
    • [SIM] Subscribed to openhab/actuator/Heat_SP_Room1_fmu
    • [SIM] Subscribed to openhab/actuator/Heat_SP_Corridor_fmu
    • [SIM] Subscribed to openhab/actuator/Heat_SP_Bath_fmu
    • [SIM] Subscribed to openhab/actuator/Heat_SP_Livingroom_fmu
    • [SIM] Subscribed to openhab/actuator/Heat_SP_Bath1_fmu
    • [SIM] Subscribed to openhab/actuator/Heat_SP_Room_fmu
► [SIM] fmu initialized, first observation: [24.00472885 24.10908049 24.01623524 24.34895093 24.13518165 24.24220078
 24.06422864 20.2       ]
[09:10:01] [SIM] Command → Heat_SP_Room1_fmu = 16.5
[09:10:02] [SIM] Command → Heat_SP_Corridor_fmu = 16.5
[09:10:03] [SIM] Command → Heat_SP_Bath_fmu = 19.0
[09:10:04] [SIM] Command → Heat_SP_Livingroom_fmu = 19.0
[09:10:05] [SIM] Command → Heat_SP_Bath1_fmu = 16.0
[09:10:07] [SIM] Command → Heat_SP_Room_fmu = 16.0
[09:10:24] [SIM] Command → Heat_SP_Room1

In [6]:
# CELDA 2: Principal simulation loop (MQTT listener y publshing)

STEP_SIZE = dfmu['fmu_step_size']
T_FINAL   = dfmu['fmu_final_time']

# Mapping each name for each action
action_idx = { name: idx for idx, name in enumerate(action_names) }

t = 0.0
obs = initial_obs

while t < T_FINAL:

    # 2) Contructing action vector
    action_vector = np.zeros(len(action_names), dtype=float)
    for name in action_names:
        idx = action_idx[name]
        val = latest_actions.get(name)
        action_vector[idx] = val if (val is not None) else action_min[idx]

    # 3) Dictionary converion and publishing actions
    action_dict = { name: float(action_vector[action_idx[name]]) for name in action_names }
    pub.publish_actions(action_dict, timestamp=env.fmu_time)

    # 4) advance simulation
    obs, reward, done, info = env.step(action_vector)

    # 5)Obserbation publishing
    obs_dict = { name: float(obs[i]) for i, name in enumerate(dfmu['observation_names']) }
    pub.publish_observations(obs_dict, timestamp=env.fmu_time)

    # 6) Advancing sim time
    t += STEP_SIZE
    time.sleep(0.005)

    if done:
        break

print("► [SIM] Simulation completed a t =", t, "seconds.")

► [SIM] Simulation completed a t = 9417600.0 seconds.


In [7]:
# CELDA 4: Closing MQTT conecctions 

pub.close()
client.loop_stop()
client.disconnect()
print("► [SIM] Mqtt conections terminated. Simulation ended.")

► [SIM] Mqtt conections terminated. Simulation ended.
